In [ ]:
from os import environ

environ['HF_HOME'] = '/Volumes/DATA/hf_home'
# environ["HF_HOME"] = "/mnt/workspaces/hf_home"
environ["HTTP_PROXY"] = "http://127.0.0.1:7890"
environ["HTTPS_PROXY"] = "http://127.0.0.1:7890"
environ["OPENAI_API_BASE"] = "http://10.1.3.28:8990/v1"
environ["OPENAI_API_KEY"] = "sk-"


from retrieval_qa_benchmark.models import *
from retrieval_qa_benchmark.datasets import *
from retrieval_qa_benchmark.transforms import *
from retrieval_qa_benchmark.protocols import *
from retrieval_qa_benchmark.utils.registry import REGISTRY
from retrieval_qa_benchmark.utils.profiler import PROFILER

print(str(REGISTRY))

In [2]:
import yaml
from retrieval_qa_benchmark.utils.config import load

config = load(open("../config/mmlu-llama2-remote-retrieval-myscale.yaml"))
# config = load(open("../config/test.yaml"))
evaluator = REGISTRY.Evaluators["mcsa"].from_config(config)
print(evaluator.transform)

In [ ]:
import json
from tqdm import tqdm
from multiprocess.pool import ThreadPool
from retrieval_qa_benchmark.utils.profiler import PROFILER

PROFILER.clear()

evaluator.transform.nodes["0"].num_selected = 10
print(evaluator.transform)


def single(i):
    return evaluator.transform(evaluator.dataset[i])


data = []
for r in tqdm(map(single, range(len(evaluator.dataset))), total=len(evaluator.dataset)):
    data.append(r)
with open("old_aligned.jsonl", "w") as f:
    f.write("\n".join([json.dumps(d.model_dump()) for d in data]))
evaluator.dataset.eval_set = evaluator.dataset.eval_set[:5]
# data[0]

In [ ]:
print(evaluator.llm.convert_record(data[0]))

In [ ]:
print(str(PROFILER))
PROFILER.clear()

In [ ]:
matched = evaluator()

In [ ]:
print(str(PROFILER))

In [ ]:
with open("results.with-retrieval.test.jsonl", "w") as f:
    f.write("\n".join([r.model_dump_json() for r in matched[1]]))

In [ ]:
for dname, dset in REGISTRY.Datasets.items():
    print(f"Loading {dname}...")
    dset.build()

In [ ]:
from retrieval_qa_benchmark.schema import QAPrediction

mismatched = [pred for pred in matched[1] if pred.matched]

In [ ]:
with open("results.test.jsonl", "w") as f:
    f.write("\n".join([r.model_dump_json() for r in mismatched]))

In [ ]:
import json

with open("old_aligned.jsonl") as f1:
    old_result = [json.loads(l) for l in f1.readlines()]

with open("new.jsonl") as f2:
    new_result = [json.loads(l) for l in f2.readlines()]

In [ ]:
from hashlib import sha256

cnt = 0
for r1, r2 in zip(old_result, new_result):
    c1 = set(map(lambda x: sha256(x.encode("utf-8")).hexdigest(), r1["context"]))
    c2 = set(map(lambda x: sha256(x.encode("utf-8")).hexdigest(), r2["context"]))
    cnt += len(c2.intersection(c1))
print(cnt / len(new_result) / 5)

In [ ]:
import json

with open("old_aligned.jsonl") as f1:
    old_result = [json.loads(l) for l in f1.readlines()]

with open("new.jsonl") as f2:
    new_result = [json.loads(l) for l in f2.readlines()]

In [ ]:
from hashlib import sha256

cnt = 0
for r1, r2 in zip(old_result, new_result):
    c1 = set(map(lambda x: sha256(x.encode("utf-8")).hexdigest(), r1["context"]))
    c2 = set(map(lambda x: sha256(x.encode("utf-8")).hexdigest(), r2["context"]))
    cnt += len(c2.intersection(c1))
print(cnt / len(new_result) / 5)

In [ ]:
#!/usr/bin/python3

import sys

if __name__ == '__main__':
    for line in sys.stdin:
        line = 
        print("Value " + line, end='')
        sys.stdout.flush()